# Project 3

Required libraries imported below

In [1]:
import matplotlib as plt
import numpy as np
import random

Data Generation

In [3]:
def is_dangerous(color_choices):
    if 3 in color_choices and 1 not in color_choices:
        return True
    return False

def one_hot_encode(color):
    arr = np.zeros(4)    
    arr[color-1] = 1
    return arr
    
def generate_wiring_diagram(part): #part signfies the part of the project the data is being generated for (1 or 2)
    if part != 1 and part != 2:
        print("invalid parameter values entered while calling generate_wiring_diagram(part) function, part can either be 1 or 2")
        return
    
    diagram = np.zeros((20,20,4), dtype=int) # Initializing a 20x20 blank diagram
    color_choices = [1, 2, 3, 4] # 1: Red, 2: Blue, 3: Yellow, 4: Green
    row_choices = list(range(0,20))
    col_choices= list(range(0,20))
    dangerous = False
    wire_to_cut = [0,0,0,0]

    start_with_row = random.choice([True, False])
    
    if start_with_row:
        for i in range(4):
            if i%2 == 0:
                row = random.choice(row_choices)
                color = random.choice(color_choices)
                diagram[row, :] = one_hot_encode(color)
                row_choices.remove(row)
                color_choices.remove(color)
                if not dangerous:
                    dangerous = is_dangerous(color_choices)
            else:
                col = random.choice(col_choices)
                color = random.choice(color_choices)
                diagram[:, col] = one_hot_encode(color)
                col_choices.remove(col)
                color_choices.remove(color)
                if i == 1 and not dangerous:
                    dangerous = is_dangerous(color_choices)
                    
            if dangerous and i==3:
                wire_to_cut = one_hot_encode(color)
                
    else:
        for i in range(4):
            if i%2 == 0:
                col = random.choice(col_choices)
                color = random.choice(color_choices)
                diagram[col, :] = one_hot_encode(color)
                col_choices.remove(col)
                color_choices.remove(color)
                if not dangerous:
                    dangerous = is_dangerous(color_choices)
            else:
                row = random.choice(row_choices)
                color = random.choice(color_choices)
                diagram[:, row] = one_hot_encode(color)
                row_choices.remove(row)
                color_choices.remove(color)
                if i == 1 and not dangerous:
                    dangerous = is_dangerous(color_choices)
                    
            if dangerous and i==3:
                wire_to_cut = one_hot_encode(color)

    if part == 1: 
        return diagram, dangerous
    elif part == 2:
        return diagram, wire_to_cut
        


Memory Class to store dataset

In [4]:
# Creating a class to create our "dataset" for model training
class Memory:
    # Constructor
    def __init__(self, max_memory):
        self.max_memory = max_memory # maximum amount of samples to remember
        self.samples = [] # the samples

    # Adding a sample to memory
    def add_sample(self, diagram_vector, y):
        self.samples.append((diagram_vector, y))
        if len(self.samples) > self.max_memory: # Removing the earliest sample if we reach maximum memory
            self.samples.pop(0)

    # Sampling the samples we have in memory
    def sample(self, no_samples): #no_samples is the number of samples you need
        if no_samples > len(self.samples):
            return random.sample(self.samples, len(self.samples))
        else:
            return random.sample(self.samples, no_samples)

Data generation for part 1

In [5]:
diagrams = 5000 #number of max diagrams to be stored in Memory class
memory = Memory(diagrams) #more than diagrams and the oldest diagram stored will be removed
for i in range(diagrams):
    diagram, y = generate_wiring_diagram(1) #type:ignore
    memory.add_sample(diagram, int(y == True))
list1 = memory.sample(500)
print(list1[0][1])
arr1 = (list1[0])[0].flatten()
arr2 = (list1[0])[0].reshape(-1,4)
"""print(arr1)
print(arr2)
for i in arr1:
    print(i, end = "")
print()
for i in arr2:
    print(i, end = "")"""


0


'print(arr1)\nprint(arr2)\nfor i in arr1:\n    print(i, end = "")\nprint()\nfor i in arr2:\n    print(i, end = "")'

Part 2 Building the Model

In [13]:
input_features = np.column_stack((np.ones(len(arr2)), arr2))
weights = np.random.randn(len(input_features[0]))    
alpha = 0.05 # Left at 0.05 for now
y = np.empty(len(list1), dtype=int)

for x in range(len(y)):
    y[x] = list1[x][1]

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def log(x):
    return np.log10(x)

def costFunction (features, weights, y): #Loss Function
    loss = 0
    for i in range(len(weights)):
        loss += -1 * y[i] * log(sigmoid(np.dot(weights[i], features[i])) - (1-y[i]) * log(sigmoid(1-np.dot(weights[i], features[i]))))
    loss /= 1/len(weights-1)
    return loss

def gradientDescent(features, weights, alpha, y):
    for i in range(len(weights)):
        gradient = 0
        for j in range(len(weights)):
            func = sigmoid(np.dot(features[j], weights))
            gradient += (func-y[i]) * features[j][i]
        weights[i] = weights[i] - alpha *  gradient

def trainModel(features, weights, alpha):
    epochs = 200
    for x in range(epochs):
        gradientDescent(features, weights, alpha, y)

def testModel(features, weights, alpha, y):
    prediction = sigmoid(np.dot(features, weights))
    print(prediction)
    print(costFunction(features, weights, y))

trainModel(input_features, weights, alpha)

testModel([1, 0, 1, 0, 0], weights, alpha, y)

0.03575761411928173
0.91102791816224


Data generation for part 2

In [33]:
diagrams = 5000 #number of max diagrams to be stored in Memory class
memory = Memory(diagrams) #more than diagrams and the oldest diagram stored will be removed
for i in range(diagrams):
    diagram, y = generate_wiring_diagram(2) #type:ignore
    memory.add_sample(diagram, y)
list1 = memory.sample(500)


print((list1[0])[0]) #printing first diagram
print((list1[0])[1]) #printing what wire to cut
print(len(list1)) #printing length of dataset to be used

[[0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0]
 [0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0]
 [0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0]
 [4 2 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 4]
 [0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0]
 [0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0]
 [0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0]
 [0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0]
 [0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0]
 [0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0]
 [0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0]
 [0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0]
 [0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0]
 [0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0]
 [0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0]
 [0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0]
 [1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0]
 [0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0]
 [0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0]]
0
500


Part 2

